# Data Processing using Pyspark

In [ ]:
#configuración en google colab de spark y pyspark
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#instalar java y spark
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [ ]:
spark

In [ ]:
sc

In [ ]:
# Load csv Dataset 
#df=spark.read.csv('s3://<bucket/dir>/sample_data.csv',inferSchema=True,header=True)
df=spark.read.csv('/content/gdrive/MyDrive/st0263-2266/bigdata/datasets/covid19/Casos_positivos_de_COVID-19_en_Colombia-100K.csv',inferSchema=True,header=True)

In [ ]:
#columns of dataframe
df.columns

In [ ]:
#printSchema
df.printSchema()

In [ ]:
df.select('fecha reporte web','Fecha de notificación','Ubicación del caso','Fecha de muerte').show(5)

In [ ]:
df.select('Fecha de notificación','Recuperado','Edad').withColumn("Edad despues de 10 años",(df["Edad"]+10)).show(10,False)

In [ ]:
df.select('ID de caso','Fecha de notificación','Recuperado','Edad').drop('Fecha de notificación').show(5)

In [ ]:
df.filter(df['Edad']>30).show(5)

In [ ]:
df.select('Nombre departamento','Recuperado').withColumnRenamed('Recuperado','sano').show(5)

In [ ]:
from pyspark.sql.functions import udf

In [ ]:
from pyspark.sql.types import StringType,DoubleType,IntegerType

In [ ]:
def realizar_cuarentena(dep):
  if dep == "Recuperado":
    return "No cuarentena"
  else:
    return "Realizar cuarentena"

In [ ]:
obj_udf=udf(realizar_cuarentena,StringType())
df.select('ID de caso','Edad','Recuperado').withColumn('Realizar cuarentena',obj_udf(df['Recuperado'])).show(10)

In [ ]:
#Respuestas a las preguntas de la sección 3

In [75]:
#Dirección 
write_uri1='s3a://datamazapataj/Preguntas3/3_1'

In [ ]:
#Departamentos con mayor casos de covid (dataframe)

df.groupBy('Nombre departamento').count().orderBy('count',ascending=False).show(10,False)

In [ ]:
#Municipio con mayor casos de covid (dataframe)

df.groupBy('Nombre municipio').count().orderBy('count',ascending=False).show(10,False)

In [ ]:
#Fechas en las que hay mayor casos de covid (dataframe)

df.groupBy('Fecha de notificación').count().orderBy('count',ascending=False).show(10,False)

In [ ]:
#Distribución de casos por edades (dataframe)

df.groupBy('Edad').count().orderBy('Edad',ascending=True).show()

In [ ]:
#Cuanta gente se ha recuperado, cuanta ha fallecido (dataframe)

df.groupBy('Recuperado').count().orderBy('count').show()

In [ ]:
#SparkSQL para las preguntas 3

In [ ]:
# In Python
from pyspark.sql import SparkSession
# Create a SparkSession
spark = (SparkSession
  .builder
  .appName("SparkSQLExampleApp")
  .getOrCreate())
# Path to data set
csv_file = "/content/gdrive/MyDrive/st0263-2266/bigdata/datasets/covid19/Casos_positivos_de_COVID-19_en_Colombia-100K.csv"
# Read and create a temporary view
# Infer schema (note that for larger files you 
# may want to specify the schema)
df = (spark.read.format("csv")
  .option("inferSchema", "true")
  .option("header", "true")
  .load(csv_file))
df.createOrReplaceTempView("DbCovid")

In [ ]:
#Distribución de casos por edades (SparkSQL)
spark.sql("""SELECT Edad, COUNT(*) FROM DbCovid GROUP BY Edad""").show(10)

In [ ]:
#Municipio con mayores casos de covid (SparkSQL)
spark.sql("""SELECT `Nombre municipio`, COUNT(*) FROM DbCovid GROUP BY `Nombre municipio` ORDER BY 2  DESC""").show(10)

In [ ]:
#Departamentos con mayores casos de covid (SparkSQL)
spark.sql("""SELECT `Nombre departamento`, COUNT(*) FROM covid GROUP BY `Nombre departamento` ORDER BY 2  DESC""").show(10)

In [ ]:
#Fechas con mayores casos de covid (SparkSQL)
spark.sql("""SELECT `Fecha de notificación`, COUNT(*) FROM covid GROUP BY `Fecha de notificación` ORDER BY 2  DESC""").show(10)

In [ ]:
#Cuanta gente se ha recuperado, cuanta ha fallecido (SparkSQL)
spark.sql("""SELECT Recuperado, COUNT(*) FROM covid GROUP BY Recuperado ORDER BY 2  DESC""").show(10)